In [ ]:
!pip install kaggle
!pip install faker
import pandas
import os
os.environ['KAGGLE_USERNAME']="nithyasri0201"
os.environ['KAGGLE_KEY']="00f24d5ce8cdb4c9c4ded4f8e55b06a7"
!kaggle datasets download -d mohammadtalib786/retail-sales-dataset
!unzip /content/retail-sales-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/mohammadtalib786/retail-sales-dataset
License(s): CC0-1.0
  0% 0.00/11.2k [00:00<?, ?B/s]
100% 11.2k/11.2k [00:00<00:00, 21.1MB/s]
Archive:  /content/retail-sales-dataset.zip
  inflating: retail_sales_dataset.csv  


In [ ]:
import pandas as pd

# Load the dataset (assuming the data is in a CSV file)
file_path = '/content/retail_sales_dataset.csv'  # Path to the dataset
df = pd.read_csv(file_path)

# Convert the Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Extract useful time-related features
df['Hour'] = df['Date'].dt.hour
df['Day'] = df['Date'].dt.dayofweek

# Display the first few rows of the dataframe
df.head()


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Hour,Day
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,0,4
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,0,0
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,0,4
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,0,6
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,0,5


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker

# Initialize Faker
fake = Faker()

# Define product categories
product_categories = ['Beauty', 'Clothing', 'Electronics', 'Groceries', 'Toys', 'Books']

# Generate synthetic data
synthetic_data = []
num_transactions = 1000

# Counter for customer IDs
customer_counter = 1

for i in range(num_transactions):
    transaction_id = i + 1
    num_items = np.random.randint(1, 5)  # Each transaction has 1 to 4 items
    selected_categories = np.random.choice(product_categories, num_items, replace=False)
    for category in selected_categories:
        quantity = np.random.randint(1, 4)  # Quantity between 1 and 3
        price_per_unit = np.random.randint(5, 500)
        customer_id = f'CUST{customer_counter:03}'  # Format customer ID as CUST001, CUST002, etc.
        synthetic_data.append({
            'Transaction ID': transaction_id,
            'Date': fake.date_this_year(),
            'Customer ID': customer_id,
            'Gender': np.random.choice(['Male', 'Female']),
            'Age': np.random.randint(18, 70),
            'Product Category': category,
            'Quantity': quantity,
            'Price per Unit': price_per_unit,
            'Total Amount': quantity * price_per_unit
        })
        customer_counter += 1

# Convert to DataFrame
synthetic_df = pd.DataFrame(synthetic_data)

# Display the first few rows of the synthetic dataset
synthetic_df.head()


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2024-01-28,CUST001,Male,58,Beauty,2,163,326
1,1,2024-03-08,CUST002,Male,44,Clothing,3,296,888
2,1,2024-01-28,CUST003,Female,56,Toys,2,230,460
3,2,2024-06-01,CUST004,Male,67,Toys,1,388,388
4,2,2024-01-03,CUST005,Male,41,Books,2,75,150


In [ ]:
# Assuming 'df' is your original dataset
# Combine the original dataset with the synthetic dataset
combined_df = pd.concat([df, synthetic_df], ignore_index=True)

# Display the combined dataset
combined_df.head()


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Hour,Day
0,1,2023-11-24 00:00:00,CUST001,Male,34,Beauty,3,50,150,0.0,4.0
1,2,2023-02-27 00:00:00,CUST002,Female,26,Clothing,2,500,1000,0.0,0.0
2,3,2023-01-13 00:00:00,CUST003,Male,50,Electronics,1,30,30,0.0,4.0
3,4,2023-05-21 00:00:00,CUST004,Male,37,Clothing,1,500,500,0.0,6.0
4,5,2023-05-06 00:00:00,CUST005,Male,30,Beauty,2,50,100,0.0,5.0


**MARKET BASKET ANALYSIS**

In [ ]:
# Create a basket for each transaction
basket = combined_df.groupby(['Transaction ID', 'Product Category'])['Quantity'].sum().unstack().reset_index().fillna(0)
basket.set_index('Transaction ID', inplace=True)

# Encode the quantities to 1 and 0
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# Display the basket
basket.head()


Product Category,Beauty,Books,Clothing,Electronics,Groceries,Toys
Transaction ID,,,,,,
1,1,0,1,0,0,1
2,0,1,1,0,0,1
3,0,1,0,1,1,0
4,1,1,1,0,0,0
5,1,0,0,1,0,0


In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Apply Apriori algorithm
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

print(frequent_itemsets)



    support                                           itemsets
0     0.594                                           (Beauty)
1     0.413                                            (Books)
2     0.607                                         (Clothing)
3     0.609                                      (Electronics)
4     0.413                                        (Groceries)
..      ...                                                ...
56    0.012  (Clothing, Electronics, Books, Beauty, Groceries)
57    0.014       (Toys, Clothing, Electronics, Books, Beauty)
58    0.015      (Toys, Electronics, Books, Beauty, Groceries)
59    0.014   (Toys, Clothing, Electronics, Beauty, Groceries)
60    0.018    (Toys, Clothing, Electronics, Books, Groceries)

[61 rows x 2 columns]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
print(rules)

                           antecedents                         consequents  \
0                               (Toys)                             (Books)   
1                              (Books)                              (Toys)   
2                        (Electronics)                         (Groceries)   
3                          (Groceries)                       (Electronics)   
4                   (Clothing, Beauty)                         (Groceries)   
5                          (Groceries)                  (Clothing, Beauty)   
6                   (Books, Groceries)                       (Electronics)   
7                        (Electronics)                  (Books, Groceries)   
8                 (Electronics, Books)                              (Toys)   
9                  (Electronics, Toys)                             (Books)   
10                       (Toys, Books)                       (Electronics)   
11                       (Electronics)                       (To

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd

# Create DataFrame
df = pd.DataFrame(rules)

# Set minimum support threshold
min_support = 0.15

# Filter rules based on minimum support
filtered_df = df[df['support'] >= min_support]

# Display filtered DataFrame
print(filtered_df)


     antecedents    consequents  antecedent support  consequent support  \
0         (Toys)        (Books)               0.430               0.413   
1        (Books)         (Toys)               0.413               0.430   
2  (Electronics)    (Groceries)               0.609               0.413   
3    (Groceries)  (Electronics)               0.413               0.609   

   support  confidence      lift  leverage  conviction  zhangs_metric  
0    0.179    0.416279  1.007940  0.001410    1.005618       0.013819  
1    0.179    0.433414  1.007940  0.001410    1.006026       0.013419  
2    0.258    0.423645  1.025776  0.006483    1.018470       0.064266  
3    0.258    0.624697  1.025776  0.006483    1.041826       0.042807  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
